In [1]:
import pandas as pd
import numpy as np

from itertools import combinations 

import sys
sys.path.append('')
sys.path.append('../')

In [2]:
from src.preprocessing.read_sql_data import get_df_from_sql
from src.preprocessing.pd_reformat import date_time_reformat, filter_outliers, group_deals_by_session_id
from src.get_session_embeddings import get_session_embeddings
from src.metrics import compute_metrics
from src.util import get_combination_idx, create_df_with_metric_values
from src.plot import plot_multi_lines_chart

/Users/diat.lov/miniconda3/envs/ml/lib/python3.8/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


### Задача: провести кластерный анализ похожих торговых сессий


In [3]:
df = get_df_from_sql()
df = date_time_reformat(df)
df = filter_outliers(df, only_monthly_data=True)
df = group_deals_by_session_id(df)

В данной работе будем рассматривать только `месячные сессии`. Выгрузим данные из базы и сгруппируем по торговым сессиям, предварительно очистив их от выбросов. 

Под `выбросами` будем понимать торговые сессии, которые длились `дольше 1 часа`. 

Далее, переведем данные о сделках в удобный для чтения формат и в столбце `deal_min` сохраним минуты, в которые совершались сделки.

In [4]:
df.head(3)

,session_id,deal_min,price,deal_id,trading_type,platform_id
0,27,"[4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, ...","[0.00682203087507895, 0.01350203734554141, 0.0...","[53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 6...",monthly,1
1,28,"[12, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[0.023059252872859655, 0.0182250593795048, 0.0...","[111, 112, 113, 114, 115, 116, 117, 118, 119, ...",monthly,2
2,29,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, ...","[0.0341719863365715, 0.11774982993404828, 0.01...","[192, 193, 194, 195, 196, 197, 198, 199, 200, ...",monthly,2


Получили `48 сессий`. Так как цены в грфе `price` представленны в виде положительных и отрицательных чисел (видимо, в зависимости от продажы или покупки), а так же вариируются в зависимости от размера сделки – приведем все числа к модулю и разделим на `lot_size`, чтобы получить цену за единицу товара.

In [5]:
monthly_session_embeddings, monthly_emb_df = get_session_embeddings(df, daily=False)

Создадим векторное представление для каждой месячной сессии. Каждую сессию представим в виде `60 чисел`, где число соответсвует цене сделки. Получившиеся эмбеддинги нормализуем в диапазон `от 0 до 1`.

In [6]:
monthly_metrics = compute_metrics(monthly_session_embeddings)
monthly_combination_idx = get_combination_idx(df)
monthly_df_metrics = create_df_with_metric_values(monthly_metrics, monthly_combination_idx)

Рассчитаем значения 3 метрик:

    - cosine similarity;
    - mre;
    - dts;
    
для всех возможных пар сочетаний из наших 48 сессий. И отсортируем по-очередно по каждой метрике.

In [21]:
monthly_df_metrics.sort_values('cos_dist', ascending=False).head(10)

,session_id_one,session_id_two,cos_dist,mre,dtw
182,35,36,1.000000,2.483527e-09,6.664002e-08
1040,174,182,0.999824,1.962096e-03,1.410199e-02
1052,175,182,0.999676,2.389220e-03,1.905675e-02
1037,174,175,0.999563,2.591706e-03,2.012724e-02
1091,182,278,0.999418,3.403733e-03,1.953099e-02
1089,182,268,0.999358,3.168793e-03,3.582709e-02
1049,174,278,0.999304,3.695761e-03,2.771560e-02
1059,175,268,0.999255,4.183098e-03,3.235138e-02
1061,175,278,0.999197,4.681868e-03,3.844371e-02
1047,174,268,0.999163,4.094884e-03,3.408189e-02


In [16]:
# df example:
df = pd.DataFrame({'time': range(60),
                   'session_174': monthly_emb_df[monthly_emb_df.session_id == 174].session_embedding.to_list()[0],
                   'session_182': monthly_emb_df[monthly_emb_df.session_id == 182].session_embedding.to_list()[0],
                   })
plot_multi_lines_chart(df, 'time', ['session_174', 'session_182'])

In [22]:
monthly_df_metrics.sort_values('mre', ascending=True).head(10)

,session_id_one,session_id_two,cos_dist,mre,dtw
182,35,36,1.000000,2.483527e-09,6.664002e-08
1040,174,182,0.999824,1.962096e-03,1.410199e-02
1052,175,182,0.999676,2.389220e-03,1.905675e-02
1037,174,175,0.999563,2.591706e-03,2.012724e-02
1089,182,268,0.999358,3.168793e-03,3.582709e-02
1091,182,278,0.999418,3.403733e-03,1.953099e-02
1041,174,183,0.998443,3.407790e-03,5.461805e-02
1027,173,182,0.998984,3.544114e-03,3.514535e-02
1049,174,278,0.999304,3.695761e-03,2.771560e-02
1023,173,174,0.998687,3.999576e-03,4.477969e-02


In [29]:
df = pd.DataFrame({'time': range(60),
                   'session_111': monthly_emb_df[monthly_emb_df.session_id == 111].session_embedding.to_list()[0],
                   'session_177': monthly_emb_df[monthly_emb_df.session_id == 177].session_embedding.to_list()[0],
                   })
plot_multi_lines_chart(df, 'time', ['session_111', 'session_177'])

In [23]:
monthly_df_metrics.sort_values('dtw', ascending=True).head(10)

,session_id_one,session_id_two,cos_dist,mre,dtw
182,35,36,1.000000,2.483527e-09,6.664002e-08
1040,174,182,0.999824,1.962096e-03,1.410199e-02
1052,175,182,0.999676,2.389220e-03,1.905675e-02
1091,182,278,0.999418,3.403733e-03,1.953099e-02
1037,174,175,0.999563,2.591706e-03,2.012724e-02
1036,173,278,0.998084,4.506050e-03,2.741509e-02
1049,174,278,0.999304,3.695761e-03,2.771560e-02
1059,175,268,0.999255,4.183098e-03,3.235138e-02
982,166,182,0.998918,4.158832e-03,3.240792e-02
979,166,175,0.998931,4.003438e-03,3.255824e-02


In [33]:
df = pd.DataFrame({'time': range(60),
                   'session_173': monthly_emb_df[monthly_emb_df.session_id == 173].session_embedding.to_list()[0],
                   'session_175': monthly_emb_df[monthly_emb_df.session_id == 175].session_embedding.to_list()[0],
                   })
plot_multi_lines_chart(df, 'time', ['session_173', 'session_175'])